In [1]:
import pandas as pd
from time import sleep
from tqdm import tqdm
#from itertools import compress

In [2]:
TARGET_YEAR = '2021'

In [3]:
def check_YYYY(YYYY):
    is_good = isinstance(YYYY, str) and len(YYYY) == 4 and YYYY.isdigit()
    if(not(is_good)):
       raise Exception("Bad args")
    return(is_good)

In [4]:
def clean_names(old_cols):
    new_cols = [c.lower().replace('#', '') for c in old_cols]
    return(new_cols)

In [5]:
def get_events(YYYY):
    check_YYYY(YYYY)
    url = f"https://rusa.org/cgi-bin/resultsearch_PF.pl?regid=&date={YYYY}&type=&dist=&rtid=&esortby=cert&collapse=0"
    print(url)
    
    # get versions with and without links
    dat_links = pd.read_html(url, extract_links='all')[0]
    dat = pd.read_html(url)[0]
    
    # break apart tuples to get urls. we'll have columns called 0 and 1
        # TODO: validate it contains Route and select columns
    route_ids = pd.DataFrame(dat_links[('Route', None)].tolist(), 
                             index=dat_links.index)
    event_ids = pd.DataFrame(dat_links[('select', None)].tolist(), 
                             index=dat_links.index)

    # extract ids from link urls in column 1
        # TODO validate regex, in case format changes
    dat['rt_url'] = route_ids[1]
    dat['rtid'] = route_ids[1].str.extract('^.*rtid=(.*)')
    dat['event_url'] = event_ids[1]
    dat['eid'] = event_ids[1].str.extract('^.*eid=(.*)&')
    
    # clean col names
    new_cols = [c.lower() for c in list(dat.columns)]
    dat.columns = new_cols

    # drop first row and the 'select' column
    dat = dat.drop(0, axis=0)
    dat = dat.drop('select', axis=1)

    return(dat)

In [6]:
events = get_events(TARGET_YEAR)
events.head()

https://rusa.org/cgi-bin/resultsearch_PF.pl?regid=&date=2021&type=&dist=&rtid=&esortby=cert&collapse=0


,region,club,type,distance,date,route,rt_url,rtid,event_url,eid
1,GA: Atlanta,Audax Atlanta / 910004,ACPB,200,2021/01/01,Athens Georgia 200k,/cgi-bin/routesearch_PF.pl?rtid=1141,1141,/cgi-bin/resultsearch_PF.pl?eid=11955&esortby=...,11955
2,VA: Tidewater,Tidewater Randonneurs / 946002,RUSAP,100,2021/01/01,Suffolk 100K,/cgi-bin/routesearch_PF.pl?rtid=2034,2034,/cgi-bin/resultsearch_PF.pl?eid=12064&esortby=...,12064
3,TX: Dallas,Lone Star Randonneurs / 943026,RUSAP,100,2021/01/01,Three Hour Tour,/cgi-bin/routesearch_PF.pl?rtid=2581,2581,/cgi-bin/resultsearch_PF.pl?eid=11988&esortby=...,11988
4,TX: Dallas,Lone Star Randonneurs / 943026,ACPB,200,2021/01/01,Rio Vista Rumble 200,/cgi-bin/routesearch_PF.pl?rtid=122,122,/cgi-bin/resultsearch_PF.pl?eid=11987&esortby=...,11987
5,TX: Austin,Hill Country Randonneurs / 943025,ACPB,200,2021/01/02,No Return to Fitzhugh 200K,/cgi-bin/routesearch_PF.pl?rtid=1722,1722,/cgi-bin/resultsearch_PF.pl?eid=12034&esortby=...,12034


In [7]:
def get_event_details_results(eid):
    
    event_url = f"https://rusa.org/cgi-bin/resultsearch_PF.pl?eid={eid}&esortby=cert"
    #print(event_url)
    two_dats = pd.read_html(event_url)
    
    event_details = two_dats[0]
    event_details.columns = clean_names(list(event_details.columns))
    event_details['eid'] = eid

    # warn if we only get one table back
    if(len(two_dats)<2):
        #print(f'Warning: no finisher results for event {eid}!')
        event_results = pd.DataFrame()
    else:
        event_results = two_dats[1]
        event_results.columns = clean_names(list(event_results.columns))
        # drop the rows 'x nonmembers also finished'
        try:
            event_results = event_results.drop(
                event_results[event_results.cert.str.contains('also finished')].index
            )
        except:
            pass
            #print(f"eid {eid} - couldn't drop 'non member' result rows")
        event_results['eid'] = eid
    
    return([event_details,event_results])

In [8]:
events.groupby('type').size()

type
ACPB     427
ACPF       9
ACPR       1
RM         5
RUSAB    130
RUSAF     10
RUSAP    143
dtype: int64

In [9]:
# https://rusa.org/pages/eventtypes
non_team_types = ['ACPB', 'RM', 'RUSAB', 'RUSAP']

In [10]:
# list of event ids for the non team event types, for which we'll get results/details
event_ids = events[events.type.isin(non_team_types)].eid.to_list()
print(len(event_ids))

705


In [11]:
e_detail_list = []
e_results_list = []

In [12]:
for e in tqdm(event_ids):
    #print(e)
    this_d_e = get_event_details_results(e)
    e_detail_list.append(this_d_e[0])
    e_results_list.append(this_d_e[1])
    # sleep roughly every sixth call (assuming event ids are consecutive ints)
    if(int(e) % 6 == 0):
        #print('Sleeping')
        sleep(1)

100%|█████████████████████████████████████████| 705/705 [06:34<00:00,  1.79it/s]


In [13]:
# should all be the same
print(len(e_detail_list), len(e_results_list), len(event_ids))

705 705 705


In [14]:
event_results = pd.concat(e_results_list)

In [15]:
event_details = pd.concat(e_detail_list)

In [16]:
# for when i forgot to filter team events before getting results; 
# the format is different and messes stuff up, plus i don't care about those results right now
# erl2 = list(compress(e_results_list, 
#                      events.type.isin(non_team_types)))

In [17]:
event_details.reset_index(inplace=True, drop=True)
event_results.reset_index(inplace=True, drop=True)

In [18]:
event_results.head()

,cert,rusa,name,club / acp code,time,medal,eid
0,RUSA-P17405,11640,"SPROCK, John P",Tidewater Randonneurs / 946002,05:53,NaN,12064
1,RUSA-P17403,8958,"HARE, Karen",Lone Star Randonneurs / 943026,04:51,NaN,11988
2,RUSA-P17404,2483,"SCHAAF, Daniel A",Lone Star Randonneurs / 943026,04:51,NaN,11988
3,768168,390,"DRISCOLL, Dan",Lone Star Randonneurs / 943026,09:22,NaN,11987
4,768169,1742,"KAISER, Christopher",Lone Star Randonneurs / 943026,12:16,NaN,11987


In [19]:
event_details.head()

,region,club,type,distance,date,finishers,dnf,eid
0,GA: Atlanta,Audax Atlanta / 910004,ACPB,200,2021/01/01,0,NaN,11955
1,VA: Tidewater,Tidewater Randonneurs / 946002,RUSAP,100,2021/01/01,1,0.0,12064
2,TX: Dallas,Lone Star Randonneurs / 943026,RUSAP,100,2021/01/01,2,0.0,11988
3,TX: Dallas,Lone Star Randonneurs / 943026,ACPB,200,2021/01/01,3,0.0,11987
4,TX: Austin,Hill Country Randonneurs / 943025,ACPB,200,2021/01/02,4,0.0,12034


In [20]:
event_details.to_csv(f'data/event_details_{TARGET_YEAR}.csv', index=False)

In [21]:
event_results.to_csv(f'data/event_results_{TARGET_YEAR}.csv', index=False)

In [22]:
events.to_csv(f'data/events_{TARGET_YEAR}.csv', index=False)